In [1]:
import pandas as pd
from nltk.corpus import inaugural
import re

In [2]:
df = pd.read_csv('./PresidentialDataFrame.csv', index_col=0)

In [3]:
#not used
df.drop('Wiki References', axis=1, inplace=True)

In [4]:
df.tail()

,First Year of Presidency,President Name,Wiki Content,Num Wiki References,Words on Page
39,1989,George H. W. Bush,"George Herbert Walker Bush (born June 12, 1924...",344,88238
40,1993,Bill Clinton,"William Jefferson ""Bill"" Clinton (born William...",392,73712
41,2001,George W. Bush,"George Walker Bush (born July 6, 1946) is an A...",787,88626
42,2009,Barack Hussein Obama,Barack Hussein Obama II ( ( listen); born Augu...,882,84498
43,2017,Donald Trump,"Donald John Trump (born June 14, 1946) is the ...",726,88048


In [5]:
#accessing the inaugural addresses of all Presidents
addresses = {}
for file in inaugural.fileids():
    if file in ['1825-Adams.txt', '1933-Roosevelt.txt', '1937-Roosevelt.txt', '1941-Roosevelt.txt', '1945-Roosevelt.txt',
               '2001-Bush.txt', '2005-Bush.txt',  '1889-Harrison.txt', '1965-Johnson.txt']:
        name = file[5:-4] + '2'
        if name in addresses:
            addresses[name] += list(inaugural.words(file))
        else:
            addresses[name] = list(inaugural.words(file))
    else:
        name = file[5:-4]
        if name in addresses:
            addresses[name] += list(inaugural.words(file))
        else:
            addresses[name] = list(inaugural.words(file))

In [6]:
#Missing Trump's first and Obama's second- adding them here
obama = open('./obama2.txt').read()
obama = obama.split()
addresses['Obama'].extend(obama)

trump = open('./trump_in.txt').read()
trump = trump.split()
addresses['Trump'] = trump

In [7]:
#getting last names to merge on
last_names = [val.split()[-1] for val in df['President Name']]
cleaned = []
for val in last_names:
    if val not in cleaned:
        cleaned.append(val)
    else:
        val = val + '2'
        cleaned.append(val)

In [8]:
df['for_merge'] = cleaned

In [9]:
#making a df to merge
address = pd.DataFrame()
address['Inaugural'] = list(addresses.values())
address['name'] = list(addresses.keys())

In [10]:
#Presidents that never delivered an inaugural address
set(df['for_merge']) - set(address['name'])

{'Arthur', 'Buren', 'Fillmore', 'Ford', 'Johnson', 'Tyler'}

In [11]:
df = pd.merge(df, address, how='left', left_on='for_merge', right_on='name')

In [12]:
#dropping redundant name columns
df.drop(['name', 'for_merge'], axis=1, inplace=True)